In [33]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import os
import sys
print(sys.version)

pd.set_option('max_columns', 100)

#website for help 
#https://medium.com/@smehta/scrape-and-create-your-own-beautiful-dataset-from-sports-reference-com-using-beautifulsoup-python-c26d6920684e

3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
url = 'https://www.pro-football-reference.com/teams/kan/2017.htm'
page = urlopen(url).read()

In [3]:
soup = bs(page)

In [4]:
table = soup.find_all("table")

In [5]:
table = table[1]

In [24]:
years = list(range(2005,2020))

In [25]:
url = 'https://www.pro-football-reference.com/teams/kan/2005.htm'

In [26]:
#predf['season'] = year
#rows = table.find_all('tr')


for year in years:
    if year == years[0]:
        predf = dict()
        week = 0
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df1 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df1.iterrows():
            aa.append(index + 1)
        df1['week'] = aa
        df1['season'] = year
    else:
        predf = dict()
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df2 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df2.iterrows():
            aa.append(index + 1)
        df2['week'] = aa
        df2['season'] = year
        df1 = pd.concat([df1,df2],axis = 0,ignore_index=True)
            

In [27]:
#rearrange columns so season and week are first
df1 = pd.concat([df1[['season','week']],df1.iloc[:,:-2]],axis = 1)

In [28]:
df1[df1['season'] == 2015]

,season,week,game_day_of_week,game_date,game_time,boxscore_word,game_outcome,overtime,team_record,game_location,...,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st
176,2015,1,Sun,September 13,1:02PM ET,boxscore,W,,1-0,@,...,97,,24,396,298,98,2,1.05,4.01,2.24
177,2015,2,Thu,September 17,8:29PM ET,boxscore,L,,1-1,,...,147,5,21,299,238,61,1,-9.40,6.63,-5.00
178,2015,3,Mon,September 28,8:30PM ET,boxscore,L,,1-2,@,...,75,1,26,448,325,123,,8.10,-16.14,0.93
179,2015,4,Sun,October 4,1:03PM ET,boxscore,L,,1-3,@,...,113,1,18,445,321,124,,4.58,-23.79,5.41
180,2015,5,Sun,October 11,1:02PM ET,boxscore,L,,1-4,,...,117,,21,328,241,87,1,-0.04,2.95,-6.04
181,2015,6,Sun,October 18,1:02PM ET,boxscore,L,,1-5,@,...,57,1,16,321,237,84,2,-7.81,3.56,0.93
182,2015,7,Sun,October 25,1:03PM ET,boxscore,W,,2-5,,...,138,,18,339,192,147,3,8.55,2.64,-0.95
183,2015,8,Sun,November 1,9:36AM ET,boxscore,W,,3-5,,...,206,,16,276,195,81,2,21.11,10.04,4.13
184,2015,9,,,,,,,,,...,,,,,,,,,,
185,2015,10,Sun,November 15,4:30PM ET,boxscore,W,,4-5,@,...,106,,18,221,152,69,5,-4.23,14.78,7.40


In [29]:
df1['playoff_game'] = 'no'
aa = []
for index, row in df1.iterrows():
    if row['game_location'] == '@':
        aa.append('away')
    else:
        aa.append('home')
df1['game_location'] = aa

In [30]:
for index, row in df1.iterrows():
    if row['week'] > 17:
        df1.loc[index,'week'] = df1.loc[index,'week'] - 1
        df1.loc[index,'playoff_game'] = 'yes'
        

In [31]:
df1 = df1[df1['game_date'] != 'Playoffs']
del[df1['boxscore_word']]

In [35]:
df1[df1['game_location'] == 'home'].tail(10)

,season,week,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,playoff_game
259,2019,6,Sun,October 13,1:00PM ET,L,,4-2,home,Houston Texans,24,31,20,309,256,53,2,35,472,280,192,3,6.10,-17.92,1.87,no
261,2019,8,Sun,October 27,8:20PM ET,L,,5-3,home,Green Bay Packers,24,31,21,337,249,88,1,21,374,256,118,,11.40,-17.67,0.68,no
262,2019,9,Sun,November 3,1:00PM ET,W,,6-3,home,Minnesota Vikings,26,23,17,377,230,147,1,17,308,212,96,,7.56,-3.06,-0.23,no
265,2019,12,,,,,,,home,Bye Week,,,,,,,,,,,,,,,,no
266,2019,13,Sun,December 1,4:25PM ET,W,,8-4,home,Oakland Raiders,40,9,22,259,163,96,,19,332,210,122,3,14.17,3.44,11.19,no
268,2019,15,Sun,December 15,1:00PM ET,W,,10-4,home,Denver Broncos,23,3,27,419,327,92,1,15,251,199,52,1,17.31,9.67,-4.87,no
270,2019,17,Sun,December 29,1:00PM ET,W,,12-4,home,Los Angeles Chargers,31,21,16,336,174,162,1,25,366,258,108,2,13.59,-9.18,7.29,no
272,2019,18,Sun,January 12,3:05PM ET,W,,13-4,home,Houston Texans,51,31,29,434,316,118,1,23,442,348,94,1,28.73,-5.46,-6.77,yes
273,2019,19,Sun,January 19,3:05PM ET,W,,14-4,home,Tennessee Titans,35,24,27,404,292,112,,20,295,210,85,,29.52,-5.92,-9.99,yes
274,2019,20,Sun,February 2,6:30PM ET,W,,15-4,home,San Francisco 49ers,31,20,26,397,268,129,2,21,351,210,141,2,18.54,-5.19,-1.58,yes
